# Using data from the cloud

````{admonition} To-do
**Activate your virtual environment and start a jupyterlab session**. For example, if your mamba/conda environment is called ```12860_env```, enter the following lines in a command window (the first line activates the virtual environment and the second line starts the jupyter session):
```bash
> conda activate 12860_env
> jupyter lab
```
**Then, execute the cells in the rest of this notebook** (you can either copy/paste code to a blank notebook, or download this notebook directly).
````

## Packages

In [ ]:
import pathlib
import numpy as np
import xarray as xr
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime
import seaborn as sns
import cmocean

## (optional) remove gridlines from plots
sns.set(rc={"axes.facecolor": "white", "axes.grid": False})

## Open ERSST data

In [ ]:
## load from NOAA website
sst = xr.open_dataset(
    r"http://psl.noaa.gov/thredds/dodsC/Datasets/noaa.ersst.v5/sst.mnmean.nc"
)

## rename coordinates for consistency with other scripts
sst = sst["sst"].rename({"lon": "longitude", "lat": "latitude"})

```{note}
If you get a ```NetCDF: HDF``` error, see the [FAQ page](../resources/FAQ.md).
```

## Plot a sample

### Define functions for plotting

In [ ]:
def plot_setup(fig, projection, lon_range, lat_range):
    """Add a subplot to the figure with the given map projection
    and lon/lat range. Returns an Axes object."""

    ## Create subplot with given projection
    ax = fig.add_subplot(projection=projection)

    ## Subset to given region
    extent = [*lon_range, *lat_range]
    ax.set_extent(extent, crs=ccrs.PlateCarree())

    ## draw coastlines
    ax.coastlines()

    return ax


def plot_setup_pacific(fig):
    """Plot Pacific region"""

    ## set map projection to orthographic
    proj = ccrs.Orthographic(central_longitude=180, central_latitude=50)

    return plot_setup(fig, proj, lon_range=[120, 240], lat_range=[30, 70])


def plot_setup_atlantic(fig):
    """Plot Pacific region"""

    ## set map projection to orthographic
    proj = ccrs.Orthographic(central_longitude=-50, central_latitude=40)

    return plot_setup(fig, proj, lon_range=[-100, 0], lat_range=[10, 70])

### Make the plot

Select which time index to plot

In [ ]:
## specify time index to plot
t_idx = 0

## get year and month for labeling
year = sst.time.dt.year.values
month = sst.time.dt.month.values

Create the plot

In [ ]:
## blank canvas to plot on
fig = plt.figure(figsize=(8, 3))

## draw background map
ax = plot_setup_atlantic(fig)

## plot the first timestep of 2m-temperature
plot_data = ax.contourf(
    sst.longitude,
    sst.latitude,
    sst.isel(time=0),
    transform=ccrs.PlateCarree(),
    levels=np.arange(-2, 30, 2),
    cmap="cmo.thermal",
    extend="both",
)

## create colorbath
colorbar = fig.colorbar(plot_data, label=r"$^{\circ}$C")

## Mark Woods Hole on map
ax.scatter(
    288.5,
    41.5,
    transform=ccrs.PlateCarree(),
    marker="*",
    c="magenta",
    s=150,
    zorder=10,
)

## label
ax.set_title(f"SST ({year[t_idx]}-{month[t_idx]:02d})")

plt.show()

## Compute Woods Hole "climate index"

### Write function to compute the index from global data

In [ ]:
def get_WH_idx(sst):
    """
    Input: 'sst' is a 3-D DataArray (lon,lat,time)

    Output: 'climate_index' is a 1-D DataArray (time)
    """

    ## Get temperature in grid cell near Woods Hole
    sst_idx = sst.sel(latitude=41.5, longitude=288.5, method="nearest").compute()

    ## Remove monthly mean (climatology)
    sst_idx_clim = sst_idx.groupby("time.month").mean()
    sst_idx = sst_idx.groupby("time.month") - sst_idx_clim

    return sst_idx

### Compute the index

In [ ]:
sst_WH = get_WH_idx(sst).compute()

### Plot the result

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3))

## Plot the data
ax.plot(sst_WH.time, sst_WH, lw=0.7)

## add line at y=0
ax.axhline(0, ls="--", c="k", lw=1)

## label axes
ax.set_title(r"Woods Hole SST anomaly")
ax.set_ylabel(r"$K$")

## label x-axis
ax.set_xticks([datetime.date(y, 1, 1) for y in [1850, 1935, 2020]])
ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y"))

plt.show()